### 풀링레이어, 드롭아웃레이어 추가해서 퍼포먼스 높히기

In [1]:
# Fashion MNIST 데이터셋 불러오기 및 정규화
import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist
(train_X, train_Y),(test_X, test_Y) = fashion_mnist.load_data()

train_X = train_X / 255
test_X = test_X / 255

4422102/4422102 [==============================] - 0s 0us/step


In [2]:
# before reshape
print(f"train_X의 shape : {train_X.shape}, test_X의 shape : {test_X.shape}")

# 데이터의 차원 맨 뒤에 채널 정보 추가(Fashion_MNIST 데이터셋의 이미지는 흑백이므로 1을 추가한다.)
train_X = train_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)

# reshape 수행후의 데이터 shape 살펴보기
print(f"train_X의 shape : {train_X.shape}, test_X의 shape : {test_X.shape}")

train_X의 shape : (60000, 28, 28), test_X의 shape : (10000, 28, 28)
train_X의 shape : (60000, 28, 28, 1), test_X의 shape : (10000, 28, 28, 1)


In [3]:
# Google Colab GPU 사양 확인
!nvidia-smi

Sun Dec 11 12:51:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# Fashion MNIST 분류 컨볼루션 신경망 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(28,28,1), kernel_size=(3,3), filters=32),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(rate=0.3),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 3, 3, 128)         73856     
                                                                 
 flatten_1 (Flatten)         (None, 1152)              0

In [7]:
# Fashion MNIST 분류 컨볼루션 신경망 모델 학습 - 풀링 레이어, 드롭아웃 레이어 추가
history = model.fit(train_X, train_Y, epochs=25, validation_split=0.25)

Epoch 1/25
1407/1407 [==============================] - 15s 5ms/step - loss: 0.5280 - accuracy: 0.8100 - val_loss: 0.3624 - val_accuracy: 0.8679
Epoch 2/25
1407/1407 [==============================] - 6s 5ms/step - loss: 0.3636 - accuracy: 0.8702 - val_loss: 0.3218 - val_accuracy: 0.8822
Epoch 3/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.3215 - accuracy: 0.8844 - val_loss: 0.3294 - val_accuracy: 0.8779
Epoch 4/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.2906 - accuracy: 0.8927 - val_loss: 0.3617 - val_accuracy: 0.8742
Epoch 5/25
1407/1407 [==============================] - 6s 5ms/step - loss: 0.2725 - accuracy: 0.9004 - val_loss: 0.3036 - val_accuracy: 0.8907
Epoch 6/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.2555 - accuracy: 0.9066 - val_loss: 0.2980 - val_accuracy: 0.8933
Epoch 7/25
1407/1407 [==============================] - 6s 4ms/step - loss: 0.2396 - accuracy: 0.9121 - val_loss: 0.2933 - val_accuracy

In [8]:
model.evaluate(test_X, test_Y, verbose=0)

[0.46630921959877014, 0.8980000019073486]